In [1]:
from datetime import timedelta
import math
import random

from pyClarion import (Event, Agent, Priority, Input, Pool, Choice, 
    ChunkStore, BaseLevel, Family, NumDict, Atoms, Atom, Chunk, ks_crawl)

In [2]:
"""Keyspace Definition"""

class Char(Atoms):
    """Characters for CVC and numbers."""
    A: Atom; B: Atom; C: Atom; D: Atom; E: Atom; F: Atom; G: Atom; H: Atom
    I: Atom; J: Atom; K: Atom; L: Atom; M: Atom; N: Atom; O: Atom; P: Atom
    Q: Atom; R: Atom; S: Atom; T: Atom; U: Atom; V: Atom; X: Atom; X: Atom
    Y: Atom; Z: Atom 
    _0: Atom; _1: Atom; _2: Atom; _3: Atom; _4: Atom; _5: Atom; _6: Atom; 
    _7: Atom; _8: Atom; _9: Atom


class IO(Atoms):
    """IO sort for dimension symbols"""
    cvc1: Atom
    cvc2: Atom
    cvc3: Atom
    num1: Atom
    num2: Atom


class PairedAssoc(Family):
    io: IO
    char: Char

In [4]:
"""Model Construction"""

class Participant(Agent):  
    d: PairedAssoc 
    input: Input
    store: ChunkStore
    blas: BaseLevel
    pool: Pool
    choice: Choice

    def __init__(self, name: str) -> None:
        p = Family()
        e = Family()
        d = PairedAssoc()
        super().__init__(name, p=p, e=e, d=d)
        self.d = d
        with self:
            self.input = Input("input", (d, d))
            self.store = ChunkStore("store", d, d, d)
            self.blas = BaseLevel("blas", p, e, self.store.chunks)
            self.pool = Pool("pool", p, self.store.chunks, func=NumDict.sum)
            self.choice = Choice("choice", p, self.store.chunks)
        self.store.bu.input = self.input.main
        self.blas.input = self.choice.main
        self.pool["store.bu"] = (
            self.store.bu.main, 
            lambda d: d.shift(x=1).scale(x=0.5).logit())
        self.pool["blas"] = (
            self.blas.main, 
            lambda d: d.bound_min(x=1e-8).log().with_default(c=0.0))
        self.choice.input = self.pool.main
        with self.pool.params[0].mutable():
            self.pool.params[0][~self.pool.p["blas"]] = 2e-1
        self.blas.ignore.add(~self.store.chunks.nil)

    def resolve(self, event: Event) -> None:
        if event.source == self.store.bu.update:
            self.blas.update()
        if event.source == self.blas.update:
            self.choice.trigger()

    def start_trial(self, 
        dt: timedelta, 
        priority: Priority = Priority.PROPAGATION
    ) -> None:
        self.system.schedule(self.start_trial, dt=dt, priority=priority)

    def finish_trial(self, 
        dt: timedelta, 
        priority: Priority = Priority.PROPAGATION
    ) -> None:
        self.system.schedule(self.finish_trial, dt=dt, priority=priority)

In [ ]:
"""Knowledge Initialization"""

def init_stimuli(d: PairedAssoc, l: list[str]) -> list[tuple[Chunk, Chunk]]:
    io, char = d.io, d.char
    return [
        (s ^
         (query :=
         s[:3] ^ 
         + io.CVC1 ** char[s[0]]
         + io.CVC2 ** char[s[1]]
         + io.CVC3 ** char[s[2]])
         + io.NUM1 ** char[f"_{s[4]}"]
         + io.NUM2 ** char[f"_{s[5]}"],
         query) 
        for s in l]



In [ ]:
"""Event Processing"""

participant = Participant("participant")
stimuli = init_stimuli(participant.d, ["BAC_23", "DEB_54", "VUJ_96", "XUP_24"])
indices = list(range(len(stimuli))) * 2 
random.shuffle(indices)
presentations = {}
trial = 0
results = {
    "trial": [],
    "stim": [],
    "time": [],
    "delta": [],
    "response": [],
    "correct": [],
    "strength": [],
    "rt": []
}
participant.start_trial(timedelta())
while participant.system.queue:
    event = participant.system.advance()
    print(event.describe())
    if event.source == participant.start_trial:
        i = indices[trial]
        study, test = stimuli[i]
        if i not in presentations:
            participant.store.compile(study)
        else:
            participant.input.send(test)
        participant.finish_trial(timedelta(seconds=3))
    if event.source == participant.finish_trial:
        i = indices[trial]
        if i in presentations:
            target, _ = stimuli[i]
            time = participant.system.clock.time / timedelta(seconds=1)
            response_key = participant.choice.poll()[~participant.store.chunks]
            response_chunk = ks_crawl(participant.system.root, response_key)
            results["trial"].append(trial)
            results["stim"].append(target._name_)
            results["time"].append(time)
            results["delta"].append(trial - presentations[i])
            results["response"].append(response_chunk._name_) # type: ignore
            results["correct"].append(response_chunk == target)
            results["strength"].append(participant.choice.sample[0][response_key])
            results["rt"].append(math.exp(-participant.choice.sample[0][response_key]))
        else:
            presentations[i] = trial
        if trial < 2 * len(stimuli) - 1:
            participant.start_trial(timedelta(seconds=2))
            trial += 1

event 0x0000 00:00:00.00 064 0 participant.start_trial
event 0x0000 00:00:00.00 096 1 store.compile
event 0x0000 00:00:00.00 096 3 store.update_buw
event 0x0000 00:00:00.00 096 4 blas.invoke
event 0x0000 00:00:03.00 064 2 participant.finish_trial
event 0x0000 00:00:05.00 064 5 participant.start_trial
event 0x0000 00:00:05.00 096 6 store.compile
event 0x0000 00:00:05.00 096 8 store.update_buw
event 0x0000 00:00:05.00 096 9 blas.invoke
event 0x0000 00:00:08.00 064 7 participant.finish_trial
event 0x0000 00:00:10.00 064 10 participant.start_trial
event 0x0000 00:00:10.00 096 11 store.compile
event 0x0000 00:00:10.00 096 13 store.update_buw
event 0x0000 00:00:10.00 096 14 blas.invoke
event 0x0000 00:00:13.00 064 12 participant.finish_trial
event 0x0000 00:00:15.00 064 15 participant.start_trial
event 0x0000 00:00:15.00 096 16 store.compile
event 0x0000 00:00:15.00 096 18 store.update_buw
event 0x0000 00:00:15.00 096 19 blas.invoke
event 0x0000 00:00:18.00 064 17 participant.finish_trial
ev

In [7]:
results

{'trial': [4, 5, 6, 7],
 'stim': ['DEB_54', 'XUP_24', 'BAC_23', 'VUJ_96'],
 'time': [23.0, 28.0, 33.0, 38.0],
 'delta': [2, 4, 6, 4],
 'response': ['nil', 'XUP_24', 'nil', 'VUJ_96'],
 'correct': [False, True, False, True],
 'strength': [0.9999295154840093,
  1.0280828027403066,
  -0.3667192508034325,
  1.1057792866094578],
 'rt': [0.36790537188964256,
  0.3576920698706281,
  1.4429927431881284,
  0.33095287458055805]}